# Perception Project – Uygulama 2  
## Proje Amacı, Genel Yapı ve Hedefler

Bu not defteri, **C++ + OpenCV + LibTorch** kullanarak oluşturduğumuz **PerceptionNode tabanlı algılama modülünün** _2. uygulama aşamasını_ açıklamak için hazırlanmıştır.

Bu aşamada odaklandığımız ana konu:

> “Kamera görüntüsünü (`cv::Mat`), derin öğrenme modelinin anlayacağı **doğru tensor formatına** (`torch::Tensor`) dönüştüren profesyonel bir **preprocess pipeline’ı** kurmak.”

## 1. Bu Proje Nedir?

Bu proje, gerçek zamanlı çalışan bir sistemde (örneğin:

- Otonom araç,
- Mobil robot,
- Drone,
- Güvenlik kamerası,
- Endüstriyel otomasyon hattı)

kullanılabilecek bir **algılama (Perception) modülünün iskeletini** C++ tarafında inşa etmeyi amaçlar.

Modülün **çekirdeği** şudur:

```cpp
std::vector<DetectedObject> detections = perception.process(frame);
```

Burada:

- `frame` → Kamera karesini ve bağlam bilgilerini taşıyan `ImageFrame`,
- `perception` → Konfigüre edilmiş bir `PerceptionNode` nesnesi,
- `detections` → Algılanan nesneleri tutan `std::vector<DetectedObject>`

olarak tanımlıdır.

Bu yapı sayesinde algılama işlemi:

- Tek bir fonksiyon çağrısına indirgenir (`process()`),
- İç detaylar (preprocess, model, postprocess) dışarıdan **gizlenir**,
- Node, ileride ROS2 / multi-thread / pipeline gibi yapılara kolayca entegre edilebilir hale gelir.

## 2. Bu Proje Neden Yapılıyor?

Gerçek projelerde genelde şu tür kodlarla başlanır:

- “Hızlıca OpenCV ile resmi oku,”
- “Modeli yükle, `forward` yap,”
- “Çıktıyı biraz işle, ekrana çiz.”

Bu tarz kodlar **genelde tek dosya**, **fonksiyon yığını** ve **dağınık** haldedir.  
Uzun vadede:

- Bakımı zor,
- Test edilmesi zor,
- Farklı projelerde tekrar kullanması zor,
- ROS2 / gerçek zamanlı sistemlere taşınması zor

hale gelir.

Bu proje tam olarak şunu çözmek için yapılıyor:

> “Model + OpenCV kodumu fonksiyon curcunası olmaktan çıkarıp,  
>  endüstriyel seviyede, net sorumlulukları olan, modüler bir **PerceptionNode** mimarisine çevirmek istiyorum.”

Bu sayede:

- Algılama kodu tek bir **node sınıfı** ile temsil edilir,
- Veri modelleri (ImageFrame, DetectedObject, PerceptionConfig) net bir şekilde ayrılır,
- Node, başka projelere **copy-paste** ile değil, **mimari olarak taşınabilir bir modül** olarak eklenir.

## 3. Projenin Genel Amacı

Perception mimarisi kabaca şu şekilde tasarlanmıştır:

```bash
Kamera / Video / Resim
        ↓
   ImageFrame
        ↓
  PerceptionNode
   ├─ preprocess()   → cv::Mat → torch::Tensor
   ├─ inference()    → model.forward(tensor)
   └─ postprocess()  → tensor → DetectedObject list
        ↓
std::vector<DetectedObject>
```

Genel amaçlar:

1. **Girdi tiplerini netleştirmek**  
   - `ImageFrame` ile bir kamera karesini; görüntü, id, timestamp, kamera adı gibi bilgilerle tek yerde toplamak.

2. **Çıktı tipini netleştirmek**  
   - `DetectedObject` ile her bir tespiti (sınıf id, sınıf adı, skor, bbox, track id) yüksek seviyeli bir veri modeli haline getirmek.

3. **Konfigürasyonu soyutlamak**  
   - `PerceptionConfig` ile model yolu, cihaz seçimi, input boyutu, normalize parametreleri, sınıf isimleri gibi ayarları tek bir yerde toplamak.

4. **Algılama akışını tek node’a bağlamak**  
   - `PerceptionNode` sınıfı ile preprocess, inference, postprocess gibi adımları yönetmek,
   - Dışarıya sadece `process(frame)` fonksiyonunu açmak.

5. **Gerçek zamanlı sistemlere uygun node mimarisi kurmak**  
   - İleride ROS2 node’u, multi-thread pipeline’ı, sensör füzyon yapıları bu PerceptionNode üstüne inşa edilebilsin.

## 4. Uygulama 1 – Ne Yaptık?

**Uygulama 1’in amacı mimariyi kurmaktı.**

Bu aşamada:

- `ImageFrame` tanımlandı:
```cpp
  - `cv::Mat image`
  - `std::uint64_t frame_id`
  - `std::chrono::steady_clock::time_point timestamp`
  - `std::string camera_name`
```


- `DetectedObject` tanımlandı:
```cpp
  - `class_id`, `class_name`
  - `score`
  - `x, y, width, height`
  - `track_id`
```
- `PerceptionConfig` tanımlandı:
```cpp
  - `model_path`
  - `device` (`"cpu"` / `"cuda"`)
  - `input_width`, `input_height`
  - `mean`, `std`
  - `class_names`
```

- `PerceptionNode` iskeleti yazıldı:
```cpp
  - Constructor: config alıp cihaz seçen ve modeli yüklemeye hazırlanan yapı
  - `process(frame)`: preprocess → inference → postprocess zincirini yöneten ana fonksiyon
  - `loadModel()`, `preprocess()`, `inference()`, `postprocess()` imzaları belirlendi
```

- `main.cpp` içinde:
```cpp
  - Bir `test.jpg` yüklendi,
  - `ImageFrame` oluşturuldu,
  - `PerceptionConfig` dolduruldu,
  - `PerceptionNode` ayağa kaldırıldı,
  - `perception.process(frame)` çağrıldı,
  - Gelen `DetectedObject` listesi loglandı.
```

> Uygulama 1’de içerik **dummy** idi: preprocess random tensor, inference dummy output, postprocess sahte 1 nesne üretiyordu.  
> Amaç: **“Node mimarisi doğru oturdu mu?”** sorusuna cevap almaktı.

## 5. Uygulama 2 – Ne Yapacağız?

**Uygulama 2**, bu mimariyi bozmadan, içerisinde çalışan **preprocess pipeline’ını gerçek hale getirdiğimiz aşamadır.**

Özellikle şu soruya odaklanır:

> “Bir `ImageFrame` içindeki `cv::Mat` görüntüyü, derin öğrenme modelinin beklediği formatta **doğru bir `torch::Tensor`’a dönüştürebiliyor muyum?”

Yani:

- Artık random tensor değil,
- Doğru boyutta,
- Doğru kanalda (RGB),
- Doğru skala ([0,1]), 
- Doğru normalize (mean/std),
- Doğru eksen dizilimi (NCHW),
- Doğru cihazda (CPU/CUDA)

bir giriş tensörü üretmek istiyoruz.

## 6. Uygulama 1 ve Uygulama 2 Arasındaki Temel Fark

### Uygulama 1

- Node iskeleti kuruldu,
- Her şey dummy çalıştı,
- Preprocess / inference / postprocess gerçek değildi,
- Amaç: sınıf yapısı, veri akışı, process() mantığını anlamak.

### Uygulama 2

- Aynı sınıf ve struct yapısını kullanıyoruz (hiçbir mimari bozulmuyor),
- Sadece şunu yapıyoruz:

> `preprocess(const ImageFrame& frame)` fonksiyonunu **gerçek cv::Mat → torch::Tensor pipeline’ı** olarak yeniden yazıyoruz.

Bu aşamadan sonra:

- `process()` çağrıldığında içerideki `preprocess()` gerçekten:
  - resize,
  - renk dönüşümü,
  - normalizasyon,
  - tensor formatı,
  - batch dimension,
  - device taşıma  

  adımlarını uygulamış olacak.

## 7. Uygulama 2’de Kullanılacak Teknik Adımlar

`preprocess()` fonksiyonunun hedef pipeline’ı:

1. **Görüntü boş mu kontrol et**
   - `frame.image.empty()` ise hata log’u.

2. **Resize**
   - `config_.input_width` x `config_.input_height` boyutuna getirme:
   ```cpp
   cv::resize(frame.image, resized, cv::Size(config_.input_width, config_.input_height));
   ```

3. **BGR → RGB dönüşümü**
   - OpenCV görüntüyü BGR tutar, çoğu PyTorch modeli RGB bekler:

   ```cpp
   cv::cvtColor(resized, rgb, cv::COLOR_BGR2RGB);
   ```

4. **float’a çevirme ve [0,1] aralığına alma**
   ```cpp
   rgb.convertTo(float_img, CV_32F, 1.0 / 255.0);
   ```

5. **Tensor oluşturma (HWC)**
   - `torch::from_blob` ile `cv::Mat` → `torch::Tensor`
   - İlk hali: `[H, W, C]` (HWC)

6. **HWC → CHW permute**
   ```cpp
   tensor = tensor.permute({2, 0, 1});  // [C, H, W]
   ```

7. **Batch dimension ekleme**
   ```cpp
   tensor = tensor.unsqueeze(0);        // [1, C, H, W]
   ```

8. **Normalize (mean/std)**
   - `config_.mean` ve `config_.std`:
   ```cpp
   // Örneğin: mean = {0.485, 0.456, 0.406}
   //          std  = {0.229, 0.224, 0.225}
   ```
   - Kanal bazlı `(x - mean) / std` uygulama.

9. **Doğru cihaza taşıma (CPU / CUDA)**
   ```cpp
   tensor = tensor.to(device_);
   ```

Bu adımlar tamamlandığında, `preprocess()` fonksiyonunun çıktısı:

```text
torch::Tensor  →  [1, 3, input_height, input_width]
```

formatında, derin öğrenme modeline hazır bir tensördür.

## 8. Bu Proje Yapısı Nasıl Kuruluyor?

Minimal proje iskeleti şu şekildedir:

```text
PerceptionProject/
  CMakeLists.txt
  src/
    main.cpp
    perception_types.hpp
    perception_node.hpp
    perception_node.cpp
```

### `perception_types.hpp`
- `ImageFrame`
- `DetectedObject`
- `PerceptionConfig`

→ Tüm veri modelleri burada bulunur.

### `perception_node.hpp`
- `PerceptionNode` sınıf deklarasyonu,
- Constructor, `process`, `preprocess`, `inference`, `postprocess`, `loadModel` imzaları.

### `perception_node.cpp`
- `PerceptionNode` fonksiyonlarının gerçek implementasyonu,
- Uygulama 1’de dummy içerik,
- Uygulama 2’de **gerçek preprocess** burada kodlanır.

### `main.cpp`
- Bir görüntü kaynağı seçer (şimdilik `test.jpg`),
- `ImageFrame` oluşturur,
- `PerceptionConfig` doldurur,
- `PerceptionNode` oluşturur,
- `perception.process(frame)` çağırır,
- Sonuçları log’lar ve test eder.

## 9. Bu Proje Sana Ne Kazandıracak?

Bu projeyi bu şekilde adım adım kurgulamak sana şunları kazandırır:

### 1. Node Tabanlı Düşünme

- “Algılama kodu” artık rastgele fonksiyonlar değil,
- Net sorumluluğu olan bir **PerceptionNode sınıfı**,
- Real-time / ROS2 / sensör füzyon mimarilerine uyumlu yapı.

### 2. C++’ta Derin Öğrenme Altyapısı

- LibTorch (PyTorch C++ API) ile model entegrasyonu nasıl yapılır,
- Tensor kavramı C++ tarafında nasıl yönetilir (device, shape, dtype).

### 3. Gerçek Preprocess Bilgisi

- cv::Mat → tensor dönüşümünün bütün kritik adımları:
  - Resize
  - BGR/RGB farkı
  - [0,1] skalası
  - Normalizasyon (mean/std)
  - HWC → CHW
  - Batch dimension
  - CPU / CUDA cihaza taşıma

Bu bilgiyi doğru öğrendiğinde, hangi CNN / YOLO modelini kullanırsan kullan, girişi doğru hazırlamayı bilirsin.

### 4. Yeniden Kullanılabilir Altyapı

Bu node yapısını:

- Farklı projelerde,
- Farklı modellerle,
- Farklı input kaynaklarıyla (kamera, video, dosya),
- Farklı platformlarda (PC, gömülü sistem, araç bilgisayarı)

tekrar tekrar kullanabilirsin.

### 5. Otomotiv / Savunma / Robotik Mimari Bakışı

Bu proje, tipik bir otonom sürüş stack’inde “Perception” katmanının C++ tarafındaki sade bir versiyonudur.  
Bu stili öğrenmek, ileride daha büyük sistemlerde çalışırken sana ciddi avantaj sağlar.

## 10. Özet – Uygulama 2’nin Katkısı

Uygulama 2’yi tek cümleyle şöyle özetleyebiliriz:

> “PerceptionNode mimarisini bozmadan, `preprocess()` fonksiyonunu gerçek `cv::Mat → torch::Tensor` pipeline’ına çeviriyoruz ve algılama modülünün giriş tarafını üretim kalitesine çıkarıyoruz.”

Bu aşama tamamlandığında:

- Node’un **input bacağı** profesyonel hale gelmiş olacak,
- Sıradaki aşama olan `inference()` (gerçek TorchScript model bağlama) ve `postprocess()` (bbox & class decode, NMS) için sağlam bir temel oluşacak.

---
---
---


# Şimdi proje yapısını ve kodlarını paylaşıyorum

## 1️⃣ Proje yapısı

Bu yapıyı oluşturacağız:
```bash
PreprocessProject/
  CMakeLists.txt
  src/
    main.cpp
    frame_types.hpp
    preprocess_node.hpp
    preprocess_node.cpp
```

## 2️⃣ frame_types.hpp
```cpp
#ifndef FRAME_TYPES_HPP
#define FRAME_TYPES_HPP

#include <opencv2/core.hpp>
#include <cstdint>
#include <string>

// Tek bir görüntü karesini temsil eder.
struct ImageFrame
{
    cv::Mat image;              // Ham görüntü
    std::uint64_t frame_id{0};  // Frame numarası / id
    std::string source_name;    // Kaynak adı (örn: "file:test.jpg")

    ImageFrame() = default;

    ImageFrame(const cv::Mat& img,
               std::uint64_t id,
               const std::string& source)
        : image(img),
          frame_id(id),
          source_name(source)
    {}
};

#endif // FRAME_TYPES_HPP


## 3️⃣ preprocess_node.hpp
```cpp
#ifndef PREPROCESS_NODE_HPP
#define PREPROCESS_NODE_HPP

#include "frame_types.hpp"

#include <torch/torch.h>
#include <string>
#include <vector>

// Preprocess ayarları (giriş boyutu, normalize parametreleri, cihaz)
struct PreprocessConfig
{
    int input_width{640};
    int input_height{640};

    std::vector<float> mean{0.485f, 0.456f, 0.406f};
    std::vector<float> std{0.229f, 0.224f, 0.225f};

    std::string device{"cpu"}; // "cpu" veya "cuda"
};

// Sadece preprocess işini yapan node
class PreprocessNode
{
public:
    explicit PreprocessNode(const PreprocessConfig& config);

    // Dışarıya açılan API: ImageFrame -> Tensor
    torch::Tensor process(const ImageFrame& frame);

private:
    // Asıl işi yapan iç fonksiyon
    torch::Tensor preprocess(const ImageFrame& frame);

private:
    PreprocessConfig config_;
    torch::Device device_{torch::kCPU};
};

#endif // PREPROCESS_NODE_HPP

## 4️⃣ preprocess_node.cpp
```cpp
#include "preprocess_node.hpp"

#include <opencv2/imgproc.hpp>
#include <iostream>

// Constructor: config alır ve cihaz seçer
PreprocessNode::PreprocessNode(const PreprocessConfig& config)
    : config_(config)
{
    if (config_.device == "cuda" && torch::cuda::is_available())
    {
        device_ = torch::kCUDA;
        std::cout << "[PreprocessNode] CUDA kullanılacak.\n";
    }
    else
    {
        device_ = torch::kCPU;
        std::cout << "[PreprocessNode] CPU kullanılacak.\n";
    }
}

// Dış API: şu an direkt preprocess çağırıyoruz
torch::Tensor PreprocessNode::process(const ImageFrame& frame)
{
    return preprocess(frame);
}

// Asıl cv::Mat -> torch::Tensor dönüşümü
torch::Tensor PreprocessNode::preprocess(const ImageFrame& frame)
{
    std::cout << "[preprocess] Orijinal boyut: "
              << frame.image.cols << "x" << frame.image.rows << "\n";

    // 1) Boş görüntü kontrolü
    if (frame.image.empty())
    {
        std::cerr << "[preprocess] HATA: image boş, boş tensor döndürülüyor.\n";
        return torch::empty({0});
    }

    // 2) Resize
    cv::Mat resized;
    cv::resize(frame.image,
               resized,
               cv::Size(config_.input_width, config_.input_height));

    std::cout << "[preprocess] Resize sonrası boyut: "
              << resized.cols << "x" << resized.rows << "\n";

    // 3) BGR -> RGB
    cv::Mat rgb;
    cv::cvtColor(resized, rgb, cv::COLOR_BGR2RGB);

    // 4) float'a çevir ve [0, 1] aralığına getir
    cv::Mat float_img;
    rgb.convertTo(float_img, CV_32F, 1.0 / 255.0);

    const int height   = float_img.rows;
    const int width    = float_img.cols;
    const int channels = float_img.channels(); // 3

    auto options = torch::TensorOptions()
                       .dtype(torch::kFloat32)
                       .device(torch::kCPU);   // ilk etapta CPU'da

    // 5) from_blob: OpenCV buffer'ını saran bir tensor (H, W, C)
    torch::Tensor tensor = torch::from_blob(
        float_img.data,
        {height, width, channels}, // [H, W, C]
        options
    );

    // 6) HWC -> CHW : [H, W, C] -> [C, H, W]
    tensor = tensor.permute({2, 0, 1});

    // 7) Batch dimension ekle: [C, H, W] -> [1, C, H, W]
    tensor = tensor.unsqueeze(0);

    // 8) Normalize (mean / std)
    if (config_.mean.size() == 3 && config_.std.size() == 3)
    {
        torch::Tensor mean = torch::tensor(config_.mean, options).view({1, 3, 1, 1});
        torch::Tensor std  = torch::tensor(config_.std,  options).view({1, 3, 1, 1});

        tensor = (tensor - mean) / std;
    }
    else
    {
        std::cout << "[preprocess] UYARI: mean/std boyutu 3 değil, normalize atlandı.\n";
    }

    // 9) Doğru cihaza taşı (CPU / CUDA)
    tensor = tensor.to(device_);

    // 10) from_blob OpenCV buffer'ına bağlı, güvenli olmak için clone
    tensor = tensor.clone();

    std::cout << "[preprocess] Tensor hazır. Shape: " << tensor.sizes()
              << ", device: " << (device_.is_cuda() ? "CUDA" : "CPU") << "\n";

    return tensor;
}


## 5️⃣ main.cpp
```cpp
#include <iostream>
#include <opencv2/opencv.hpp>

#include "frame_types.hpp"
#include "preprocess_node.hpp"

int main()
{
    // 1) Test görüntüsünü yükle
    cv::Mat img = cv::imread("test.jpg");
    if (img.empty())
    {
        std::cerr << "[main] test.jpg yüklenemedi. "
                  << "Çalışma dizinini ve dosya yolunu kontrol et.\n";
        return 1;
    }

    std::cout << "[main] test.jpg yüklendi. Boyut: "
              << img.cols << "x" << img.rows << "\n";

    // 2) ImageFrame oluştur
    ImageFrame frame(img, 1, "file:test.jpg");

    // 3) PreprocessConfig ayarla
    PreprocessConfig config;
    config.input_width  = 640;
    config.input_height = 640;

    // ImageNet / PyTorch standart normalize
    config.mean = {0.485f, 0.456f, 0.406f};
    config.std  = {0.229f, 0.224f, 0.225f};

    config.device = "cpu"; // CUDA kullanacaksan "cuda" yap

    // 4) Node oluştur
    PreprocessNode node(config);

    // 5) Preprocess çalıştır
    torch::Tensor input_tensor = node.process(frame);

    if (input_tensor.numel() == 0)
    {
        std::cerr << "[main] Boş tensor döndü, preprocess başarısız.\n";
        return 1;
    }

    std::cout << "[main] Final tensor shape: " << input_tensor.sizes() << "\n";
    std::cout << "[main] bitti.\n";

    return 0;
}


## 6️⃣ CMakeLists.txt
```bash
cmake_minimum_required(VERSION 3.10)
project(PreprocessProject LANGUAGES CXX)

set(CMAKE_CXX_STANDARD 17)
set(CMAKE_CXX_STANDARD_REQUIRED ON)

# LibTorch yolunu ortamında ayarlaman gerekiyor:
# cmake .. -DCMAKE_PREFIX_PATH="C:/libtorch"
find_package(Torch REQUIRED)
find_package(OpenCV REQUIRED)

add_executable(preprocess_app
    src/main.cpp
    src/frame_types.hpp
    src/preprocess_node.hpp
    src/preprocess_node.cpp
)

target_include_directories(preprocess_app PRIVATE
    ${OpenCV_INCLUDE_DIRS}
)

target_link_libraries(preprocess_app
    "${TORCH_LIBRARIES}"
    ${OpenCV_LIBS}
)

# Windows için: LibTorch DLL'lerini kopyalamak gerekebilir
if (MSVC)
    set_property(TARGET preprocess_app PROPERTY VS_DEBUGGER_ENVIRONMENT
        "PATH=%PATH%;${TORCH_INSTALL_PREFIX}/lib")
endif()


## Derleme adımı (özet)
```cpp
cd PreprocessProject
mkdir build
cd build

cmake .. -DCMAKE_PREFIX_PATH="LIBTORCH_KLASORUN"
cmake --build .
```


* test.jpg dosyasını build klasörüne veya çalışma dizinine koy, sonra:

>**./preprocess_app**


* çıktıda tensor shape ([1, 3, 640, 640] gibi) ve device loglarını görmen lazım.